In [155]:
import numpy as np
import pandas as pd
import itertools
import time
import pickle


- 先需求候補車

In [93]:
raw = pd.read_csv("youbike.csv")
raw=raw.rename(columns={'Unnamed: 1':"站點"})
raw=raw.rename(columns={'Unnamed: 2':"車柱數量"})
savedCol = ['站點', '車柱數量']
savedCol+=[str(i)+":00" for i in range(4)]
savedCol+=[str(i)+":00" for i in range(9,24)]
df = raw[savedCol][:104]
pickedStop = ['捷運公館站(2號出口)', '臺大總圖書館西南側', '臺大社科院西側']
data = {}
for stop in pickedStop:
    tmp = [int(x) for x in (df[df['站點'] == stop]).values[0][1:]]
    data[stop] = tmp
data

{'捷運公館站(2號出口)': [60,
  0,
  0,
  -27,
  4,
  -4,
  7,
  -6,
  -1,
  4,
  20,
  -22,
  15,
  -9,
  -6,
  10,
  3,
  4,
  5,
  -7],
 '臺大總圖書館西南側': [30,
  0,
  0,
  9,
  5,
  2,
  0,
  2,
  0,
  3,
  3,
  -21,
  -2,
  2,
  0,
  -3,
  0,
  0,
  0,
  0],
 '臺大社科院西側': [26,
  0,
  0,
  10,
  2,
  -4,
  15,
  0,
  -12,
  12,
  -10,
  -12,
  0,
  -1,
  0,
  2,
  -2,
  0,
  0,
  0]}

In [146]:
def minimum_cost(T, D, V, pillarNum, CostPerBike, CosrPerStop):
    for t in range(1, T + 1):
        
        lbound = int(pillarNum * (1/3))
        hbound = int(pillarNum * (2/3))
#         for y in range(pillarNum+1):
        for y in range(lbound, hbound+1):
            choices = {}
            if(D[T-t] > 0 and y-D[T-t] < lbound):#需要補車
                # 少 y-D[T-t][s] 台車
                min_x = lbound-(y-D[T-t])#至少滿足需求
                max_x = hbound - (y-D[T-t])#至多不超過上限
                choices["situation"] = "補車"
            elif(D[T-t] < 0 and y-D[T-t] > hbound):#需要移車
                #有 -D[s][t]-(hbound-y) 車沒位置停
                min_x = -D[T-t]-(hbound-y)           #至少滿足需求
                max_x = (y-D[T-t])-lbound           #至多不小魚下限
                choices["situation"] = "移車"
            else: #不用移or補
#                     print("stop",s, ": ", "no need, lbound={}, hbound={}, D={}, y={}".format(lbound, hbound, D[T-t][s], y))
                choices["situation"] = "不用去"
                min_x = max_x = 0

            choices["pillar"] = pillarNum
            choices["lbound"] = lbound
            choices["hbound"] = hbound
            choices["D"] = D[T-t]
            choices["min_x"] = min_x
            choices["max_x"] = max_x
#             print("y={}".format(y), ": ", choices)

            
            opt_x = 0
            if t == 1:
                min_cost = 9999999
                for x in range(min_x, max_x+1):
                    z = 1 if x > 0 else 0
                    cost = CostPerBike * x + CosrPerStop * z
                    if cost < min_cost: 
                        min_cost = cost
                        opt_x = x
            elif choices["situation"] == "補車": 
                min_cost = 9999999
                for x in range(min_x, max_x+1):
                    z = 1 if x > 0 else 0
                    cost = V[t - 1][y + x - D[T - t]-lbound][1] + CostPerBike * x + CosrPerStop * z 
                    if cost < min_cost: 
                        min_cost = cost
                        opt_x = x
            elif choices["situation"] == "移車": 
                min_cost = 9999999
                for x in range(min_x, max_x+1):
                    z = 1 if x > 0 else 0
                    cost = V[t - 1][y - x - D[T - t]-lbound][1] + CostPerBike * x + CosrPerStop * z 
                    if cost < min_cost: 
                        min_cost = cost
                        opt_x = x
            elif choices["situation"] == "不用去": 
                min_cost = 0
            
#             print("t={}, y={}, opt_x={}".format(t,y,opt_x))
            V[t][y-lbound] = (opt_x, min_cost)
        
        
def print_min_cost(V, head):

    print("      ",end="")
    for i in head:
        print("{:>4}".format(i), end="")
    print()
    time = ['{:2d}'.format(i)+":00" for i in range(1,4)] + ['{:2d}'.format(i)+":00" for i in range(9, 25)]
    time = list(reversed(time))
    for t in range(len(V)):
        if t == 0:
            print('{:5d}'.format(0), end=" \n")
#             continue
        else:
            print(time[t-1], end=" ")
        for y in range(len(V[0])):
            print("{:>4}".format(V[t][y][1]), end="")
            #print(V[t][y], end = " ")
        print()
    print()

In [147]:
for stop, values in data.items():
    # parameter for stop
    T = len(values) - 1
    pillarNum = values[0]
    D = values[1:]
    V = [0] * (T + 1)
    for i in range(len(V)):
        V[i] = [0] * (int(pillarNum * (2/3))- int(pillarNum * (1/3)) + 1)
    CostPerBike = 1
    CosrPerStop = 5


    minimum_cost(T, D, V, pillarNum, CostPerBike, CosrPerStop)
    head = list(range(int(pillarNum * (1/3)),int(pillarNum * (2/3))+1))
    print(stop)
    print_min_cost(V, head)

捷運公館站(2號出口)
        20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40
    0 
24:00    0   0   0   0   0   0   0   0   0   0   0   0   0   0   6   7   8   9  10  11  12
23:00   10   9   8   7   6   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
22:00   14  13  12  11   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
21:00   12  11  10   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
20:00   18  17  16  15  14  13  12  11  10   9   0   0   0   0   0   0   0   0   0   0   0
19:00    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   6   7   8   9  10  11
18:00    0   0   0   0   0   0   0   0   0   0   0   0  12  13  14  15  16  17  18  19  20
17:00   20  19  18  17  16  15  14  13  12  11  10   9   8   7   6   0   0   0   0   0   0
16:00    7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
15:00   32  31  30  29  28  27  26  25  24  23  22  21  20  19  18  17 

In [ ]:
# parameter for stop
T = 4
D = [0 ,-9, -3,  1 , 9]
pillarNum = 10
V = [0] * (T + 1)
for i in range(len(V)):
    V[i] = [0] * (int(pillarNum * (2/3))- int(pillarNum * (1/3)) + 1)
CostPerBike = 1
CosrPerStop = 5


minimum_cost(T, D, V, pillarNum, CostPerBike, CosrPerStop)
head = list(range(int(pillarNum * (1/3)),int(pillarNum * (2/3))+1))
print_min_cost(V, head)

In [157]:
all_data = []
for i in range(1,14):
    with open('20200617data/'+str(i)+".pkl", 'rb') as f:
        d = pickle.load(f)
    all_data.append(d)
    f.close()


In [167]:
pre_time = all_data[0][i]['time']
post_time = all_data[1][i]['time']
print(pre_time, "~", post_time)
for i in range(len(all_data[0])):
    stop_name = all_data[1][i]['name_tw']
    pre_num = all_data[0][i]['available_spaces']
    post_num = all_data[1][i]['available_spaces']
    print(stop_name, post_num, pre_num, post_num-pre_num)

2020-06-17 09:49:32 ~ 2020-06-17 09:56:33
捷運科技大樓站 0 0 0
復興南路二段273號前 0 0 0
國北教大實小東側門 0 0 0
和平公園東側 0 0 0
辛亥復興路口西北側 1 1 0
復興南路二段280號前 0 0 0
復興南路二段340巷口 0 0 0
新生南路三段52號前 0 0 0
新生南路三段82號前 0 0 0
羅斯福路三段333巷9號旁 0 0 0
辛亥路一段30號前 0 0 0
羅斯福路三段311號前 1 1 0
羅斯福路三段245號前 1 1 0
大安運動中心停車場 5 5 0
羅斯福路三段269巷5號前 0 0 0
溫州公園 0 0 0
和平臥龍街口 0 0 0
溫州停車場 0 0 0
銘傳國小側門 0 0 0
第二學生活動中心 1 2 -1
臺灣科技大學正門 9 9 0
臺灣科技大學側門 9 8 1
公館公園 2 2 0
臺灣科技大學後門 11 12 -1
臺大醫學院附設癌醫中心 14 14 0
臺大環研大樓 14 14 0
臺大永齡生醫工程館 7 5 2
臺大男七舍前 1 1 0
臺大男一舍前 0 1 -1
臺大男六舍前 0 0 0
臺大動物醫院前 11 12 -1
臺大土木研究大樓前 0 0 0
臺大萬才館前 9 9 0
臺大國青大樓宿舍前 0 0 0
臺大社科院圖書館前 0 1 -1
臺大法人語言訓練中心前 12 12 0
臺大綜合體育館停車場前 0 0 0
辛亥新生路口東南側 0 0 0
基隆長興路口東側 0 0 0
國北教大側門南側 0 0 0
和平實小東北側 2 2 0
大安區健康服務中心 1 1 0
臺北自來水事業處 0 0 0
汀州思源街口東南側 0 0 0
捷運公館站(1號出口) 0 0 0
捷運公館站(4號出口) 0 0 0
和平復興路口西北側 0 0 0
新生南路三段66號前 0 0 0
捷運公館站(2號出口) 0 1 -1
辛亥泰順街口 1 1 0
臺大博理館西側 9 9 0
臺大立體機車停車場 9 8 1
臺大凝態館南側 1 1 0
臺大小福樓東側 20 17 3
臺大公館停車場西北側 0 0 0
臺大總圖書館西南側 28 28 0
臺大新體育館東南側 5 6 -1
臺大明達館機車停車場 1 0 1
臺大博雅館西側 5 5 0
臺大電機二館東南側 5 5 0
臺大檔案